<a href="https://colab.research.google.com/github/Subarashiihibi/geosearch/blob/main/geosearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# created on Dec 24, 2020
# modified on April 22, 2022
# @author:          Steven Chen
# @email:           chenj274@uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [ ]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file1 = '/gdrive/My Drive/geotweets1.csv'
    output_file2 = '/gdrive/My Drive/geotweets2.csv'
    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "RVLruo4qU8TCTSnE5mZU4nGyZ"
    consumer_secret = "tcGJb2ivRUKYncEm6u2FATkLUpRkS0Z6VxQZSffekIjI4PdQK9"
    access_token = "1517218305932599296-9lzygLIwoyUhWmvpitQ1jx5swDeEaa"
    access_token_secret = "ACnv9JX3VhQi99a8o3RD9AgrkV776vqrbQbvg78Bg8arG"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATION1 = [-124.7771694, 24.520833, -66.947028, 49.384472] # Contiguous US
    LOCATION2 = [-122.961596, 46.104790, -117.033058, 48.998833,] # Washington State
    LOCATION3 = [-160.161542, 18.776344, -154.641396, 22.878623] # Hawaii

    stream_listener1 = StreamListener(time_limit=120, file=output_file1)
    stream1 = tweepy.Stream(auth=myauth, listener=stream_listener1)
    stream1.filter(locations=LOCATION1, languages=['en'], encoding="utf-8", track=["Netflix"])

    stream_listener2 = StreamListener(time_limit=120, file=output_file2)
    stream2 = tweepy.Stream(auth=myauth, listener=stream_listener2)
    stream2.filter(locations=LOCATION2, languages=['en'], encoding="utf-8", track=["Netflix"])

{'id': 1517695155574173697, 'username': 'MarkGThwaite', 'created_at': 'Sat Apr 23 02:41:37 +0000 2022', 'lng': -118.4119065, 'lat': 34.0207895, 'text': 'Just recording some tunes 🎸⚡️⚡️What are you doing this weekend?#guitarporn #burmanamps https://t.co/a4KCDtckX8'}
{'id': 1517695156572540931, 'username': 'GiftedAndGoofy', 'created_at': 'Sat Apr 23 02:41:37 +0000 2022', 'lng': -86.791549, 'lat': 36.200008, 'text': 'Big Val supposed to have 3 rebounds btw. But I understand the betting agendas'}
{'id': 1517695158049030145, 'username': 'NWJS_jobs', 'created_at': 'Sat Apr 23 02:41:37 +0000 2022', 'lng': -121.807403, 'lat': 39.753601, 'text': 'This job might be a great fit for you: Associate Director, Systems &amp; Compliance https://t.co/Ei9LQ7MqRa Chico, CA'}
{'id': 1517695158334201861, 'username': 'ChelsyCass', 'created_at': 'Sat Apr 23 02:41:37 +0000 2022', 'lng': -97.28591349999999, 'lat': 32.7799665, 'text': '@r_robertooo Omg yes tell me why I didn’t know till today because someone tol

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
{'id': 1517695684207681536, 'username': 'byAmandaOcasio', 'created_at': 'Sat Apr 23 02:43:43 +0000 2022', 'lng': -78.85356300000001, 'lat': 42.896229500000004, 'text': 'Binging this Thai show on Netflix.You could cross Jordan Peele with M. Night Shyamalan &amp; still wouldnt get anythi… https://t.co/aAUwWUp3l9'}
{'id': 1517695696626925569, 'username': 'steveconsultsp1', 'created_at': 'Sat Apr 23 02:43:46 +0000 2022', 'lng': -121.8228047, 'lat': 42.1301336, 'text': "19:34 Friday at the Midland Rest Area watching Traffic on Land and Air.  There's Alot of Geese, Ducks, Seagulls, fl… https://t.co/UuH95plGvS"}
{'id': 1517695702519795712, 'username': 'YikesMountain', 'created_at': 'Sat Apr 23 02:43:47 +0000 2022', 'lng': -117.234439, 'lat': 47.6625765, 'text': '@YetiBlanc_ @ILLBeDamnItsWes @therealmasmiami @Bo_Eighty @iamkevinito @DudeGoods @GOPispoop @RJT243… https://t.co/Wj8tNAqsnk'}
{'id': 151

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
